In [38]:
import spacy
from collections import Counter, OrderedDict, defaultdict
from datasets import load_dataset
from elasticsearch import Elasticsearch
import random
from spacy.tokenizer import Tokenizer
import pandas as pd
import re
import math

In [2]:
client = Elasticsearch(
    "http://localhost:9200",
    basic_auth=("elastic", "<hidden>"),
    verify_certs=False
)

In [5]:
ds = load_dataset("clarin-knext/fiqa-pl", "corpus")
ds_queries = load_dataset("clarin-knext/fiqa-pl","queries")
ds_qa = load_dataset("clarin-knext/fiqa-pl-qrels")['test']
nlp = spacy.load("pl_core_news_sm") # spacy language model for polish
tokenizer = Tokenizer(nlp.vocab)

### Compute frequency list for each of the processed files.

In [12]:
df=pd.DataFrame(ds['corpus'])
df['text'] = df['text'].apply(lambda text: re.sub('\W+', ' ', text))
df['tokens']=df['text'].apply(lambda x: tokenizer(x.lower()))
df.drop(columns=['title'], inplace=True)

In [13]:
df

,_id,text,tokens
0,3,Nie mówię że nie podoba mi się też pomysł szko...,"(nie, mówię, że, nie, podoba, mi, się, też, po..."
1,31,Tak więc nic nie zapobiega fałszywym ocenom po...,"(tak, więc, nic, nie, zapobiega, fałszywym, oc..."
2,56,Nigdy nie możesz korzystać z FSA dla indywidua...,"(nigdy, nie, możesz, korzystać, z, fsa, dla, i..."
3,59,Samsung stworzył LCD i inne technologie płaski...,"(samsung, stworzył, lcd, i, inne, technologie,..."
4,63,Oto wymagania SEC Federalne przepisy dotyczące...,"(oto, wymagania, sec, federalne, przepisy, dot..."
...,...,...,...
57633,599946,Cóż po pierwsze drogi to coś więcej niż hobby...,"( , cóż, po, pierwsze, drogi, to, coś, więcej,..."
57634,599953,Tak robią Na dotacje dla firm farmaceutycznych...,"(tak, robią, na, dotacje, dla, firm, farmaceut..."
57635,599966,To bardzo smutne że nie rozumiesz ludzkiej na...,"( , to, bardzo, smutne, że, nie, rozumiesz, lu..."
57636,599975,Czy Twój CTO pozwolił dużej grupie użyć admin...,"( , czy, twój, cto, pozwolił, dużej, grupie, u..."


In [14]:
def tokens_to_text_list(tokens): # without it something didn't work
    # We assume that `tokens` is a list of spacy.tokens.token.Token objects
    return [token.text for token in tokens]

df['tokens'] = df['tokens'].apply(tokens_to_text_list)

In [15]:
def count_tokens(tokens):
    return dict(Counter(tokens))

# Create new 'freq_list' column with token counts
df['freq_list'] = df['tokens'].apply(count_tokens)

In [16]:
df

,_id,text,tokens,freq_list
0,3,Nie mówię że nie podoba mi się też pomysł szko...,"[nie, mówię, że, nie, podoba, mi, się, też, po...","{'nie': 5, 'mówię': 1, 'że': 3, 'podoba': 1, '..."
1,31,Tak więc nic nie zapobiega fałszywym ocenom po...,"[tak, więc, nic, nie, zapobiega, fałszywym, oc...","{'tak': 1, 'więc': 2, 'nic': 1, 'nie': 3, 'zap..."
2,56,Nigdy nie możesz korzystać z FSA dla indywidua...,"[nigdy, nie, możesz, korzystać, z, fsa, dla, i...","{'nigdy': 1, 'nie': 2, 'możesz': 2, 'korzystać..."
3,59,Samsung stworzył LCD i inne technologie płaski...,"[samsung, stworzył, lcd, i, inne, technologie,...","{'samsung': 3, 'stworzył': 1, 'lcd': 1, 'i': 3..."
4,63,Oto wymagania SEC Federalne przepisy dotyczące...,"[oto, wymagania, sec, federalne, przepisy, dot...","{'oto': 1, 'wymagania': 1, 'sec': 1, 'federaln..."
...,...,...,...,...
57633,599946,Cóż po pierwsze drogi to coś więcej niż hobby...,"[ , cóż, po, pierwsze, drogi, to, coś, więcej,...","{' ': 1, 'cóż': 1, 'po': 2, 'pierwsze': 1, 'dr..."
57634,599953,Tak robią Na dotacje dla firm farmaceutycznych...,"[tak, robią, na, dotacje, dla, firm, farmaceut...","{'tak': 1, 'robią': 1, 'na': 1, 'dotacje': 2, ..."
57635,599966,To bardzo smutne że nie rozumiesz ludzkiej na...,"[ , to, bardzo, smutne, że, nie, rozumiesz, lu...","{' ': 1, 'to': 3, 'bardzo': 1, 'smutne': 1, 'ż..."
57636,599975,Czy Twój CTO pozwolił dużej grupie użyć admin...,"[ , czy, twój, cto, pozwolił, dużej, grupie, u...","{' ': 1, 'czy': 3, 'twój': 1, 'cto': 2, 'pozwo..."


In [19]:
global_freq_list = Counter()

# Count all dictionary elements in 'freq_list'
for freq_dict in df['freq_list']:
    global_freq_list.update(freq_dict)

global_freq_list = dict(global_freq_list)
global_freq_list = dict(sorted(global_freq_list.items(), key=lambda item: item[1], reverse=True))

In [20]:
from itertools import islice
dict(islice(global_freq_list.items(), 10))

{'w': 175696,
 'nie': 132040,
 'i': 127191,
 'na': 119136,
 'to': 117734,
 'z': 97166,
 'jest': 93567,
 'że': 90027,
 'się': 85874,
 'do': 66678}

In [23]:
queries_df = pd.DataFrame(
    [(int(row['_id']), row['text']) for row in ds_queries['queries']],
    columns=['query_id', 'text']
)
# Query filtering for test query_id and sorting
test_query_ids = set(ds_qa['query-id'])
filtered_queries_df = queries_df[queries_df['query_id'].isin(test_query_ids)].sort_values(by="query_id").reset_index(drop=True)

In [25]:
filtered_queries_df

,query_id,text
0,8,Jak zdeponować czek wystawiony na współpracown...
1,15,Czy mogę wysłać przekaz pieniężny z USPS jako ...
2,18,1 EIN prowadzący działalność pod wieloma nazwa...
3,26,Ubieganie się o kredyt biznesowy i otrzymywani...
4,34,401k Przelew po zamknięciu firmy
...,...,...
643,10979,Zamknięcie pozycji futures
644,10994,Strata netto nie podzielona przez fundusze inw...
645,11039,Spłać zadłużenie karty kredytowej lub zarób 40...
646,11054,Podatek od krótkoterminowych zysków kapitałowy...


In [26]:
def distort_query(query):
    words = query.split()
    word_to_distort = random.choice(words)

    pos = random.randint(0, len(word_to_distort) - 1)
    original_letter = word_to_distort[pos]

    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    new_letter = random.choice([c for c in alphabet if c != original_letter])
    distorted_word = word_to_distort[:pos] + new_letter + word_to_distort[pos + 1:]

    # replacing the word in the input query
    distorted_query = query.replace(word_to_distort, distorted_word, 1)
    return distorted_query

In [27]:
filtered_queries_df['distorted'] = filtered_queries_df['text'].apply(distort_query)

In [29]:
filtered_queries_df

,query_id,text,distorted
0,8,Jak zdeponować czek wystawiony na współpracown...,Jak zdeponować czek wystawiony na współpracown...
1,15,Czy mogę wysłać przekaz pieniężny z USPS jako ...,Czy mogę wysłać przekaz pieniężny z oSPS jako ...
2,18,1 EIN prowadzący działalność pod wieloma nazwa...,l EIN prowadzący działalność pod wieloma nazwa...
3,26,Ubieganie się o kredyt biznesowy i otrzymywani...,Ubieganie się o kredyt biznesowy i otrzymywani...
4,34,401k Przelew po zamknięciu firmy,401k Przelew go zamknięciu firmy
...,...,...,...
643,10979,Zamknięcie pozycji futures,Zaaknięcie pozycji futures
644,10994,Strata netto nie podzielona przez fundusze inw...,Strata netto nie podzielona przez fundgsze inw...
645,11039,Spłać zadłużenie karty kredytowej lub zarób 40...,Spłać zadłużenie karts kredytowej lub zarób 40...
646,11054,Podatek od krótkoterminowych zysków kapitałowy...,Podatek od krótkoterminowych zksków kapitałowy...


In [30]:
distorted_queries = OrderedDict(filtered_queries_df[['query_id', 'distorted']].values)

In [32]:
def search_text(index_name: str, queries: dict, k: int):

    model_results = [[] for _ in range(4)] 
    
    for query_id, query_text in queries.items():
        search_bodies = [
            {
                "query": {
                    "multi_match": {
                        "query": query_text,
                        "fields": ["with_synonyms_with_lemmatizer"]
                    }
                },
                "size": k
            },
            {
                "query": {
                    "multi_match": {
                        "query": query_text,
                        "fields": ["without_synonyms_with_lemmatizer"]
                    }
                },
                "size": k
            },
            {
                "query": {
                    "multi_match": {
                        "query": query_text,
                        "fields": ["with_synonyms_without_lemmatizer"]
                    }
                },
                "size": k
            },
            {
                "query": {
                    "multi_match": {
                        "query": query_text,
                        "fields": ["without_synonyms_without_lemmatizer"]
                    }
                },
                "size": k
            }
        ]
    
        # Wykonujemy zapytanie dla każdego analyzera i zapisujemy wyniki do odpowiedniej listy
        for i, search_body in enumerate(search_bodies):
            response = client.search(index=index_name, body=search_body)
            
            for hit in response['hits']['hits']:
                model_results[i].append({
                    'query-id': query_id,
                    'corpus-id': int(hit['_id']),
                    'analyzer': i
                })

    return model_results

In [33]:
search_results = search_text("fiqa_pl_index_v3", distorted_queries, 10)

In [36]:
def get_DCG(relevance_scores, k):
    return sum(rel / math.log(idx + 2, 2) for idx, rel in enumerate(relevance_scores[:k]))

def get_NDCG(model_results, relevant_documents, k):

    relevance_scores = [1 if doc['corpus-id'] in relevant_documents else 0 for doc in model_results]
    dcg_k = get_DCG(relevance_scores, k)

    num_docs = len(relevant_documents)
    ideal_relevance_scores = [1] * min(num_docs, k) + [0] * (k - min(num_docs, k))
    idcg_k = get_DCG(ideal_relevance_scores, k)

    ndcg_k = dcg_k / idcg_k if idcg_k > 0 else 0
    return ndcg_k

In [39]:
correct_results = defaultdict(list)

for entry in ds_qa:
    query_id = entry['query-id']
    corpus_id = entry['corpus-id']
    correct_results[query_id].append(corpus_id)

correct_list = [{'query-id': query_id, 'corpuses-id': corpuses} for query_id, corpuses in correct_results.items()]

In [40]:
ndcgs = []
k=10

for j in range(len(search_results)):
    results = []
    for i in range(len(correct_list)):
        ndcg = get_NDCG(search_results[j][i*k:(i+1)*k], correct_list[i]['corpuses-id'], k)
        results.append((correct_list[i]['query-id'], float(ndcg)))

    ndcgs.append(results)

0 - with synonyms with lemmatizer

1 - without synonyms with lemmatizer

2 - with synonyms without lemmatizer

3 - without synonyms without lemmatizer


In [41]:
for i in range(len(ndcgs)):
    avg = sum(value for _, value in ndcgs[i]) / len(ndcgs[i])
    print(f"{i}: {round(avg*100,2)} %")

0: 18.28 %
1: 18.2 %
2: 14.23 %
3: 14.16 %


Without the lemmatizer it is even a little better than in previous lab - However, in addition to introducing typos, we counted NDCG10 instead of NDCG5

In [42]:
from morfeusz2 import Morfeusz
morfeusz = Morfeusz()

In [43]:
def find_words(text):  # Finds words not in the dictionary
    analysis = morfeusz.analyse(text)
    words = []

    for result in analysis:
        if result[2][2] == 'ign':
            words.append(result[2][0])
    return words

words = []
for id, query in distorted_queries.items():
    words.extend(find_words(query))  # Use extend to add each word directly to the list

In [76]:
words[:20]

['kojto',
 'oSPS',
 'EIN',
 'ao',
 'sprzttu',
 'zairudnić',
 'podzegającej',
 'wystawioyy',
 'corp',
 'podhtku',
 'piewiędzmi',
 'gyceny',
 'zaromione',
 'nuseru',
 'EIN',
 'LLC',
 'doiyczących',
 'złc',
 'fiiansowa',
 'LLC']

In [45]:
def levenshtein_distance(s1, s2):
    len_s1, len_s2 = len(s1), len(s2)
    dp = [[0] * (len_s2 + 1) for _ in range(len_s1 + 1)]

    for i in range(len_s1 + 1):
        dp[i][0] = i
    for j in range(len_s2 + 1):
        dp[0][j] = j

    for i in range(1, len_s1 + 1):
        for j in range(1, len_s2 + 1):
            cost = 0 if s1[i - 1] == s2[j - 1] else 1
            dp[i][j] = min(dp[i - 1][j] + 1,     # deletion
                           dp[i][j - 1] + 1,     # insertion
                           dp[i - 1][j - 1] + cost)  # substitution

    return dp[len_s1][len_s2]

Following function finds the most probable correction for a word using a frequency list. max_distance is the maximum Levenshtein distance to consider for potential correction.

In [46]:
def correct_word(word, frequency_list, max_distance=2):
    possible_corrections = []
    
    # Filtering words by length similarity for efficiency
    for candidate in frequency_list:
        if abs(len(candidate) - len(word)) <= max_distance:
            distance = levenshtein_distance(word, candidate)
            if distance <= max_distance:
                possible_corrections.append((candidate, distance, frequency_list[candidate]))

    # Sorting by distance, then by frequency
    possible_corrections.sort(key=lambda x: (x[1], -x[2]))

    # Returning the most probable correction if any candidates were found
    return possible_corrections[0][0] if possible_corrections else word

In [49]:
corrected_words = []

In [50]:
for word in words:
    corrected_word = correct_word(word, global_freq_list)
    corrected_words.append(corrected_word)

In [51]:
corrected_zip = dict(zip(words, corrected_words))

In [54]:
dict(islice(corrected_zip.items(), 20))

{'kojto': 'konto',
 'oSPS': 'oSPS',
 'EIN': 'EIN',
 'ao': 'ao',
 'sprzttu': 'sprzętu',
 'zairudnić': 'zatrudnić',
 'podzegającej': 'podlegającej',
 'wystawioyy': 'wystawiony',
 'corp': 'corp',
 'podhtku': 'podatku',
 'piewiędzmi': 'pieniędzmi',
 'gyceny': 'wyceny',
 'zaromione': 'zarobione',
 'nuseru': 'numeru',
 'LLC': 'LLC',
 'doiyczących': 'dotyczących',
 'złc': 'złe',
 'fiiansowa': 'finansowa',
 'LLw': 'w',
 'Direct': 'direct'}

In [55]:
def repair_queries(queries):
    corrected_queries = queries.copy()
    for id, query in queries.items():

        dis_words = find_words(query) # searching icnorrect words in query
        text_tokens = corrected_queries[id].split()
        corrected_tokens = [corrected_zip[word] if word in dis_words else word for word in text_tokens] # replace word with corrected
        corrected_queries[id] = " ".join(corrected_tokens)

    return corrected_queries

In [56]:
corrected_queries = repair_queries(distorted_queries)

In [57]:
dict(islice(distorted_queries.items(), 10))

{8: 'Jak zdeponować czek wystawiony na współpracownika w mojej firmie na moje kojto firmowe?',
 15: 'Czy mogę wysłać przekaz pieniężny z oSPS jako firma?',
 18: 'l EIN prowadzący działalność pod wieloma nazwami firm',
 26: 'Ubieganie się o kredyt biznesowy i otrzymywanie ao',
 34: '401k Przelew go zamknięciu firmy',
 42: 'Jakie są tajniki zakupu sprzttu do pisania jako wydatki biznesowe w firmie domowej?',
 56: 'Czy przedsiębiorca może zairudnić samozatrudnionego właściciela firmy?',
 68: 'Intencje dotyczące kwoty podzegającej odliczeniu dla małych firm',
 89: 'Jak mogę wpłacić czek wystawioyy na moją firmę na moje konto osobiste?',
 90: 'Składanie osobiste e 1099s kontra s-corp biznesowy?'}

In [58]:
dict(islice(corrected_queries.items(), 10))

{8: 'Jak zdeponować czek wystawiony na współpracownika w mojej firmie na moje konto firmowe?',
 15: 'Czy mogę wysłać przekaz pieniężny z oSPS jako firma?',
 18: 'l EIN prowadzący działalność pod wieloma nazwami firm',
 26: 'Ubieganie się o kredyt biznesowy i otrzymywanie ao',
 34: '401k Przelew go zamknięciu firmy',
 42: 'Jakie są tajniki zakupu sprzętu do pisania jako wydatki biznesowe w firmie domowej?',
 56: 'Czy przedsiębiorca może zatrudnić samozatrudnionego właściciela firmy?',
 68: 'Intencje dotyczące kwoty podlegającej odliczeniu dla małych firm',
 89: 'Jak mogę wpłacić czek wystawiony na moją firmę na moje konto osobiste?',
 90: 'Składanie osobiste e 1099s kontra s-corp biznesowy?'}

In [59]:
corrected_search_results = search_text("fiqa_pl_index_v3", corrected_queries, 10)

In [61]:
# corrected_search_results 

In [60]:
ndcgs = []
k=10

for j in range(len(corrected_search_results)):
    results = []
    for i in range(len(correct_list)):
        ndcg = get_NDCG(corrected_search_results[j][i*k:(i+1)*k], correct_list[i]['corpuses-id'], k)
        results.append((correct_list[i]['query-id'], float(ndcg)))

    ndcgs.append(results)

0 - with synonyms with lemmatizer

1 - without synonyms with lemmatizer

2 - with synonyms without lemmatizer

3 - without synonyms without lemmatizer

In [62]:
for i in range(len(ndcgs)):
    avg = sum(value for _, value in ndcgs[i]) / len(ndcgs[i])
    print(f"{i}: {round(avg*100,2)} %")

0: 19.53 %
1: 19.46 %
2: 14.57 %
3: 14.56 %


Zauważamy poprawę!!!

In [63]:
def search_text_fuzzy(index_name: str, queries: dict, k: int):

    model_results = [[] for _ in range(4)] 
    
    for query_id, query_text in queries.items():
        search_bodies = [
            {
                "query": {
                    "multi_match": {
                        "query": query_text,
                        "fields": ["with_synonyms_with_lemmatizer"],
                        "fuzziness": "AUTO"
                    }
                },
                "size": k
            },
            {
                "query": {
                    "multi_match": {
                        "query": query_text,
                        "fields": ["without_synonyms_with_lemmatizer"],
                        "fuzziness": "AUTO"
                    }
                },
                "size": k
            },
            {
                "query": {
                    "multi_match": {
                        "query": query_text,
                        "fields": ["with_synonyms_without_lemmatizer"],
                        "fuzziness": "AUTO"
                    }
                },
                "size": k
            },
            {
                "query": {
                    "multi_match": {
                        "query": query_text,
                        "fields": ["without_synonyms_without_lemmatizer"],
                        "fuzziness": "AUTO"
                    }
                },
                "size": k
            }
        ]
    
        # Wykonujemy zapytanie dla każdego analyzera i zapisujemy wyniki do odpowiedniej listy
        for i, search_body in enumerate(search_bodies):
            response = client.search(index=index_name, body=search_body)
            
            for hit in response['hits']['hits']:
                model_results[i].append({
                    'query-id': query_id,
                    'corpus-id': int(hit['_id']),
                    'analyzer': i
                })

    return model_results

In [73]:
fuzzy_search_results = search_text_fuzzy("fiqa_pl_index_v3", distorted_queries, 10)

In [74]:
ndcgs = []
k=10

for j in range(len(fuzzy_search_results)):
    results = []
    for i in range(len(correct_list)):
        ndcg = get_NDCG(fuzzy_search_results[j][i*k:(i+1)*k], correct_list[i]['corpuses-id'], k)
        results.append((correct_list[i]['query-id'], float(ndcg)))

    ndcgs.append(results)

In [75]:
for i in range(len(ndcgs)):
    avg = sum(value for _, value in ndcgs[i]) / len(ndcgs[i])
    print(f"{i}: {round(avg*100,2)} %")

0: 16.43 %
1: 16.45 %
2: 14.28 %
3: 14.3 %


In [71]:
len(global_freq_list)

170051

In [72]:
dict(islice(global_freq_list.items(), 150))

{'w': 175696,
 'nie': 132040,
 'i': 127191,
 'na': 119136,
 'to': 117734,
 'z': 97166,
 'jest': 93567,
 'że': 90027,
 'się': 85874,
 'do': 66678,
 'jeśli': 52287,
 'a': 42203,
 'ale': 41350,
 'o': 38284,
 'są': 35525,
 'jak': 33699,
 'lub': 33260,
 'za': 32380,
 'aby': 31077,
 'co': 29956,
 'od': 29890,
 'może': 26450,
 'po': 25750,
 'tak': 25696,
 'dla': 25514,
 'które': 23984,
 'możesz': 23427,
 'tego': 21958,
 'tym': 21611,
 'czy': 20335,
 'ma': 20310,
 'być': 19275,
 'ponieważ': 18969,
 'przez': 17869,
 'usd': 17626,
 'tylko': 17275,
 'więc': 17201,
 'niż': 16833,
 ' ': 16739,
 'ich': 14628,
 'pieniądze': 14021,
 'więcej': 13912,
 'gdy': 13556,
 'jako': 12995,
 'pieniędzy': 12289,
 'który': 12151,
 'masz': 11776,
 'będzie': 11728,
 'mogą': 11710,
 '1': 11096,
 'firmy': 10755,
 'również': 10560,
 'akcji': 10427,
 'bardzo': 10198,
 'ci': 10030,
 'prostu': 9782,
 'nawet': 9778,
 'lat': 9522,
 'mają': 9324,
 'wiele': 9240,
 'ten': 9028,
 'te': 8803,
 'com': 8683,
 'ze': 8620,
 'kiedy':

## Draw conclusions regarding:

### The distribution of words in the corpus,

The most frequently occurring words are short, common words often used in sentence construction ("w", "nie", "i", ...). As the word frequencies decrease, we see an increase in more content-specific words, such as "pieniądze", "firma", and "akcje". It suggests that texts likely includes a lot of texts related to finance and business. 

### The performance and results of your method compared to ElasticSearch,

| Analyzer Type                     | 0 (with synonyms, with lemmatizer) | 1 (without synonyms, with lemmatizer) | 2 (with synonyms, without lemmatizer) | 3 (without synonyms, without lemmatizer) |
|-----------------------------------|------------------------------------|---------------------------------------|---------------------------------------|------------------------------------------|
| **Distorted queries**         | 18.28%                             | 18.2%                                 | 14.23%                                | 14.16%                                   |
| **Levenshtein corrected queries** | 19.53%                         | 19.46%                                | 14.57%                                | 14.56%                                   |
| **ElasticSearch's fuzzy match**   | 16.43%                         | 16.45%                                | 14.28%                                | 14.3%                                    |


The correction of typos using the Levenshtein method improved results across all analyzers, with the most significant increase observed in configurations with lemmatization. This method is effective because it addresses errors at the editorial level (i.e., spelling mistakes), which greatly facilitates matching queries to documents, especially when a lemmatizer is also applied. As a result, queries are more precise, and the search results are more relevant.

The fuzzy match proved to be the least effective for configurations with lemmatization. However, for analyzers without lemmatization, it showed a minimal increase in effectiveness. Fuzzy matching in ElasticSearch is less precise compared to Levenshtein correction, possibly due to the mechanics of fuzzy matching itself—it tends to find the "closest" matches but doesn’t consider context in the way that correction based on frequency statistics does.

The downside is that typo correction is computationally demanding and takes a significant amount of time (around 30-40 minutes). In contrast, the fuzzy match completed in approximately 2-3 minutes.

### The validity of the obtained corrections,

The correction of words is particularly effective for longer words, as there are fewer potential candidates for how a typo might be corrected. In the case of short words, they often differ by just one letter. For example, if we write "co" intending to mean "go," we won't correct this word because, despite the typo, it exists in the dictionary. Additionally, various abbreviations from other languages pose a challenge, as we may not have those words in our database and cannot determine whether a given abbreviation is legitimate or if it was meant to refer to something else.



### Ability of an LLM to fix invalid queries.

| Original                                                                 | Corrected                                                                  |
|-----------------------------------------------------------------------------------|-----------------------------------------------------------------------------|
| Jak zdeponować czek wystawiony na współpracownika w mojej firmie na moje kojto firmowe? | Jak zdeponować czek wystawiony na współpracownika w mojej firmie na moje konto firmowe? |
| Czy mogę wysłać przekaz pieniężny z oSPS jako firma?                            | Czy mogę wysłać przekaz pieniężny z SPS jako firma?                        |
| l EIN prowadzący działalność pod wieloma nazwami firm                            | EIN prowadzący działalność pod wieloma nazwami firm.                       |
| Ubieganie się o kredyt biznesowy i otrzymywanie ao                             | Ubieganie się o kredyt biznesowy i otrzymywanie AOW.                      |
| 401k Przelew go zamknięciu firmy                                                 | 401(k) Przelew przy zamknięciu firmy.                                      |
| Jakie są tajniki zakupu sprzttu do pisania jako wydatki biznesowe w firmie domowej? | Jakie są tajniki zakupu sprzętu do pisania jako wydatki biznesowe w firmie domowej? |
| Czy przedsiębiorca może zairudnić samozatrudnionego właściciela firmy?          | Czy przedsiębiorca może zaintrygować samozatrudnionego właściciela firmy?  |
| Intencje dotyczące kwoty podzegającej odliczeniu dla małych firm                 | Intencje dotyczące kwoty podżegającej odliczeniu dla małych firm.          |
| Jak mogę wpłacić czek wystawioyy na moją firmę na moje konto osobiste?          | Jak mogę wpłacić czek wystawiony na moją firmę na moje konto osobiste?     |
| Składanie osobiste e 1099s kontra s-corp biznesowy?                             | Składanie osobistego formularza 1099s kontra S-corp biznesowy?            |


As we can see, LLM did quite well. However, it is not an ideal solution, for example, in the sentence: "Czy przedsiębiorca może zairudnić samozatrudnionego właściciela firmy?" it corrected it by replacing the word with "zaintrygować." It did not take into account the context, which LLMs often can consider.